# Projeto 2: Projecao de bilheteria
Este projeto visa verificar o volume de bilheteria de 2009 ate 2022

## Bases:
Atraves do BoxOfficeMojo, obter:

1) A bilheteria no mercado norte-americano (denominado domestico) 

2) Avaliar os filmes que estiveram em primero lugar de bilheteria 

3) Quanto foi o investimento e o retorno destes filmes 

4) Qual a bilheteria dos filmes Top 10 vs Total e Top 1 vs Total 

## Chamar as Bibliotecas:
Abaixo as blibliotecas utilizadas no projeto.

problemas de instalacao da psycopg2 no mac veja:

https://stackoverflow.com/questions/20170895/mac-virtualenv-pip-postgresql-error-pg-config-executable-not-found

In [1]:
# Chamando o URL:
import requests

#ler os dados da BoxOfficeMojo:
from bs4 import BeautifulSoup 

# levar os dados capturados para banco de dados:
import psycopg2

#trabalhar com linguagem SQL:
import sqlalchemy as db

#eventuais limpeza de dados:
import re 
import pandas as pd 
import time
import numpy as np
import datetime as dt
import holidays
from math import ceil
import selenium

In [627]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
senha_pessoal = '' #Escreva sua senha pessoal aqui

## Link e API utilizados:
Para obter as informacoes, sera necessario:

Os filmes, exemplo: https://www.boxofficemojo.com/daily/2019/?view=year

Os filmes que estavam em cartaz em determinado dia: https://www.boxofficemojo.com/date/2019-09-12/

O filme: https://www.boxofficemojo.com/releasegroup/gr3957150213/

Link no IMDBPro para obter : https://pro.imdb.com/title/tt10872600?ref_=mojo_rl_summary&rf=mojo_rl_summary

Link API: https://rapidapi.com/rapidapi/api/movie-database-imdb-alternative

###  Primeiro Webscrapping: Listagem de dia
Neste primeiro web scraping obteremos os dias a serem analisados, esta lista gerara os links para cada dia analisado, onde sera possivel obter mais informacoes sobre cada um dos dias.

In [2]:
# primeiro um teste para captura as informacoes necessarias
url_teste = 'https://www.boxofficemojo.com/daily/2009/?view=year'
# Primeiro ano da serie: 2009. Este sera o valor a ser somado ao primeiro valor do for:
ano = 2009
ano_inicial = 2009
# ano atual eh 2022, assim o range_maximo serah:
range_maximo = 2022 - ano
# exemplo do dia 31 de dezembro de 2009:
link_aaaammdd = f'2009-12-31'
# exemplo do filme "Avatar":
link_id_filme_boxofficemojo = f'rl876971521'

In [3]:
response = requests.get(url_teste)
html = response.content
time.sleep(3)
soup = BeautifulSoup(html)
#print(soup.prettify())

In [4]:
url_do_ano = f'https://www.boxofficemojo.com/daily/{ano}' # verificar como parametrizar esta parte
url_do_filme = f'https://www.boxofficemojo.com/release/{link_id_filme_boxofficemojo}' # verificar como parametrizar esta parte

In [ ]:
response = requests.get(url_do_ano)
html=response.content
time.sleep(3)
soup = BeautifulSoup(html)
time.sleep(3)
#print(soup.prettify())
table_raw = soup.find('div', attrs={'id': 'table'})
table_raw1 = table_raw.find('div', attrs={'class': 'a-section imdb-scroll-table-inner'})
#print(table_raw1.prettify())
table_rows = table_raw1.find_all('tr')
#print(table_rows[1].prettify())

In [ ]:
lista_de_filmes = [[elemento.text.strip() for elemento in first_row.find_all('td')] for first_row in table_rows]
#print(lista_de_filmes)

In [ ]:
# obtido os dados de um ano (2009), sera realizado "append" dos outros anos para completar a lista
df_teste = pd.DataFrame([[elemento.text.strip() for elemento in first_row.find_all('td')] for first_row in table_rows],
            columns = ['data_ano', 'dia_semana', 'dia_do_ano', 'top10gross','perc_yd','perc_lw','release','release_num1','gross'])
        
#lista_por_ano = [[elemento.text.strip() for elemento in first_row.find_all('td')] for first_row in table_rows]

In [ ]:
#print(df_teste)

In [ ]:
len(table_rows)

In [ ]:
print(table_rows[1].prettify())

In [ ]:
# Feita a primeira lista, necessitamos chegar no link de cada uma das datas analisadas:
link_do_dia_desejado = table_rows[1].find('td', attrs={'class': 'a-text-right mojo-field-type-date_interval'}).find('a')
#print (link_do_dia_desejado)

In [ ]:
# feita a limpeza, utilizaremos get para "segurar" o link:
link_do_dia_desejado = table_rows[1].find('td', attrs={'class': 'a-text-right mojo-field-type-date_interval'}).find('a').get('href')
#print (link_do_dia_desejado.attrs)
print (link_do_dia_desejado)

In [6]:
# feita a varredura para obter o link, transformar em uma funcao:
def extrair_link_diario(tag):
    try:
        link_do_dia_desejado = tag.find('td', attrs={'class': 'a-text-right mojo-field-type-date_interval'}).find('a').get('href')
        link_do_dia_desejado_limpo1 = re.sub(r'[/][?]ref_=bo_di_table_[0-9]+','', link_do_dia_desejado) #limpar o link para que seja possivel "joins"
        link_do_dia_desejado_limpo2 = re.sub(r'/date/','', link_do_dia_desejado_limpo1) #limpar o link para que seja possivel "joins"

        return link_do_dia_desejado_limpo2
    except AttributeError:
        pass # linha zero nao tem o parametro

#visualizar = extrair_link_diario(table_rows[1])
#print(visualizar)

In [ ]:
# aqui verificamos que a linha 0 nao possui o paramtro "a-text-right mojo-field-type-date_interval"
i=0
for item in table_rows:
    print(i)
    i=i+1
    print(item.prettify())

In [ ]:
# link de cada dia que vamos analisar:
lista_dias = [extrair_link_diario(item) for item in table_rows]
df_teste['data_aaaammdd_link'] = lista_dias
df_teste

In [ ]:
print(df_teste.iloc[2]['data_aaaammdd_link'])
print(dt.datetime.strptime(df_teste.iloc[2]['data_aaaammdd_link'], '%Y-%m-%d'))

In [7]:
datas_do_ano = []
for i in df_teste['data_aaaammdd_link']:
    try:
        #data_aaaammdd = dt.date(ano, 1, 1) + dt.timedelta(days= int(df_teste.iloc[i]['dia_do_ano'])) 
        data_aaaammdd = dt.datetime.strptime(i, '%Y-%m-%d')
        datas_do_ano.append(data_aaaammdd)
        
    except TypeError:
        datas_do_ano.append(np.NaN)
        
df_teste['data_aaaammdd'] = datas_do_ano
        
print(df_teste)

NameError: name 'df_teste' is not defined

In [ ]:
#!pip3 install holidays
#import holidays
us_holidays = [str(date[0]) for date in holidays.US(years = 2009).items()]

#print(us_holidays)

#teste_feriado = [1 if str(val).split()[0] in holidays.US() else 0 for val in df_teste.iloc[:]['data_aaaammdd']]

feriado = []
 
#df_teste.iloc['data_aaaammdd']
    
for val in df_teste['data_aaaammdd']:
    try:
        teste_feriado = [1 if str(val).split()[0] in holidays.US() else 0 ]
        feriado.append(teste_feriado)
            
    except ValueError:
        feriado.append(np.NaN)

print (feriado)

df_teste['feriado'] = feriado

#dt.date(teste_feriado,'%Y,%m,%d') in us_holidays

In [8]:
def semana_do_mes(data_selecionada):
    """ Returns the week of the month for the specified date.
    """
    first_day = data_selecionada.replace(day=1)

    dom = data_selecionada.day
    adjusted_dom = dom + first_day.weekday()
    return int(ceil(adjusted_dom/7.0)) 

In [ ]:
data_teste = dt.datetime.today()
semana_do_mes(data_teste)

In [9]:
# o link do filme Top1 para que seja possivel efetuar join futuros:
def extrair_link_top1(tag):
    try:
        link_do_dia_desejado = tag.find('td', attrs={'class': 'a-text-left mojo-field-type-release mojo-cell-wide'}).find('a').get('href')
        link_do_dia_desejado_limpo1 = re.sub(r'[/][?]ref_=bo_d[a-z]_table_[0-9]+','', link_do_dia_desejado)
        link_do_dia_desejado_limpo2 = re.sub(r'/release/','', link_do_dia_desejado_limpo1)
        return link_do_dia_desejado_limpo2
    except AttributeError:
        np.NaN

In [ ]:
lista_top1 = [extrair_link_top1(item) for item in table_rows]
df_teste['top1_link'] = lista_top1
df_teste

In [10]:
def EstacaoAno(dia, mes):
    if mes in (1, 2):
        return 'INVERNO'
    elif mes == 3:
        if dia < 21:
            return 'INVERNO'
        else:
            return 'PRIMAVERA'
    elif mes in (4, 5):
        return 'PRIMAVERA'
    elif mes == 6:
        if dia < 21:
            return 'PRIMAVERA'
        else:
            return 'VERAO'
    elif mes in (7, 8):
        return 'VERAO'
    elif mes == 9:
        if dia < 21:
            return 'VERAO'
        else:
            return 'OUTONO'
    elif mes in (10, 11):
        return 'OUTONO'
    elif mes == 12:
        if dia < 21:
            return 'OUTONO'
        else:
            return 'INVERNO'

In [ ]:
print(EstacaoAno(int(df_teste['data_aaaammdd'][5].strftime("%d")), int(df_teste['data_aaaammdd'][5].strftime("%m"))))

In [ ]:
df_teste['data_aaaammdd'][5].strftime("%m")
df_teste['data_aaaammdd'][5].strftime("%d")

In [ ]:
df_teste['data_aaaammdd'][5]

In [ ]:
estacoes = []

for estacao in df_teste['data_aaaammdd']:
    try:
        qual_estacao = EstacaoAno(int(estacao.strftime("%d")), int(estacao.strftime("%m")))
        estacoes.append(qual_estacao)
            
    except ValueError:
        estacoes.append(np.NaN)

print (estacoes)

df_teste['estacoes'] = estacoes

In [ ]:
from pandas._libs.tslibs.timestamps import Timestamp
ts = Timestamp(df_teste['data_aaaammdd'][5], freq='MS')
date_time = ts.to_pydatetime()

type(pd.date_range('2018-01', '2020-05', freq='M')[0].to_pydatetime())

nt(date_time)
#data_teste = df_teste['data_aaaammdd'][5]
#print(type(data_teste))
#print(data_teste)
semana_do_mes(date_time)

In [ ]:
df_teste['data_aaaammdd'][5]

In [ ]:
semana_do_mes = []

for semana in df_teste['data_aaaammdd']:
    try:
        qual_semana = semana_do_mes(semana)
        semana_do_mes.append(qual_semana)
                
    except NameError:
        semana_do_mes.append(np.NaN)

    except TypeError :
        semana_do_mes.append(np.NaN)
        
print (semana_do_mes)

df_teste['semana_do_mes'] = semana_do_mes

In [795]:
db_server = 'postgresql'
user = 'postgres'
password = 'senha_pessoal
ip = '127.0.0.1'
db_name = 'postgres'
#schema = 'db_projeto_cinema'
engine = db.create_engine(f'{db_server}://{user}:{password}@{ip}/{db_name}')
conn = engine.connect()

SyntaxError: EOL while scanning string literal (<ipython-input-795-8d819388ada5>, line 3)

In [12]:
# juntando as etapas para todos os anos:

for j in range(range_maximo+1):
    ano_leitura = ano_inicial + j
    url_do_ano = f'https://www.boxofficemojo.com/daily/{ano_leitura}' # verificar como parametrizar esta parte
    response = requests.get(url_do_ano)
    html=response.content
    time.sleep(3)
    soup = BeautifulSoup(html)
    time.sleep(3)
    table_raw = soup.find('div', attrs={'id': 'table'})
    table_raw1 = table_raw.find('div', attrs={'class': 'a-section imdb-scroll-table-inner'})
    table_rows = table_raw1.find_all('tr')

    lista_de_filmes = [[elemento.text.strip() for elemento in first_row.find_all('td')] for first_row in table_rows]

    df = pd.DataFrame([[elemento.text.strip() for elemento in first_row.find_all('td')] for first_row in table_rows],
            columns = ['data_ano', 'dia_semana', 'dia_do_ano', 'top10gross','perc_yd','perc_lw','release','release_num1','gross'])
    
    lista_dias = [extrair_link_diario(item) for item in table_rows]
    df['data_aaaammdd_link'] = lista_dias

    datas_do_ano = []
    for i in df['data_aaaammdd_link']:
        try:
            data_aaaammdd = dt.datetime.strptime(i, '%Y-%m-%d')
            datas_do_ano.append(data_aaaammdd)
        
        except TypeError:
            datas_do_ano.append(np.NaN)

    df['data_aaaammdd'] = datas_do_ano
    
    lista_top1 = [extrair_link_top1(item) for item in table_rows]
    df['top1_link'] = lista_top1
        
    us_holidays = [str(date[0]) for date in holidays.US(years = ano_leitura).items()]
    
    feriado = []
 
    for val in df['data_aaaammdd']:
        try:
            teste_feriado = [1 if str(val).split()[0] in holidays.US() else 0 ]
            feriado.append(teste_feriado)

        except ValueError:
            feriado.append(np.NaN)

    df['feriado'] = feriado
    
    estacoes = []

    for estacao in df['data_aaaammdd']:
        try:
            qual_estacao = EstacaoAno(int(estacao.strftime("%d")), int(estacao.strftime("%m")))
            estacoes.append(qual_estacao)

        except ValueError:
            estacoes.append(np.NaN)

    df['estacao'] = estacoes

    semana_do_mes = []

    for semana in df['data_aaaammdd']:
        try:
            qual_semana = semana_do_mes(semana)
            semana_do_mes.append(qual_semana)

        except NameError:
            semana_do_mes.append(np.NaN)

        except TypeError :
            semana_do_mes.append(np.NaN)

    df['semana_do_mes'] = semana_do_mes
    
    df = df.dropna(thresh=5)
    
    try:
        df_data = pd.concat([df_data, df],ignore_index=True)
    except NameError:
        df_data = df.copy()

In [ ]:
df_data

In [ ]:
df

In [ ]:
df_data = df_data.drop_duplicates(subset = ['data_aaaammdd'])

In [ ]:
df_data.info()

In [277]:
records_to_insert = df_data[['data_aaaammdd','dia_semana', 'feriado', 'estacao', 'semana_do_mes']]
records_to_insert = records_to_insert.dropna(thresh=1)

In [278]:
records_to_insert.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4787 entries, 0 to 4786
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   data_aaaammdd  4787 non-null   datetime64[ns]
 1   dia_semana     4787 non-null   object        
 2   feriado        4787 non-null   object        
 3   estacao        4787 non-null   object        
 4   semana_do_mes  0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 224.4+ KB


In [ ]:
mask = records_to_insert ['data_aaaammdd'] == '2009-12-31'

In [ ]:
sum(mask)

In [279]:
records_to_insert.to_sql(name='tb_data', con=conn, if_exists='append', index=False, schema ='db_projeto_cinema' )

In [ ]:
# subir no banco de dados na tabela bilheteria:

#Selecionar as colunas para enviar ao banco de dados:


In [151]:
records_to_insert

,data_aaaammdd,dia_semana,feriado,estacao,semana_do_mes
0,2009-12-31,Thursday,[0],INVERNO,NaN
1,2009-12-30,Wednesday,[0],INVERNO,NaN
2,2009-12-29,Tuesday,[0],INVERNO,NaN
3,2009-12-28,Monday,[0],INVERNO,NaN
4,2009-12-27,Sunday,[0],INVERNO,NaN
...,...,...,...,...,...
4782,2022-01-05,Wednesday,[0],INVERNO,NaN
4783,2022-01-04,Tuesday,[0],INVERNO,NaN
4784,2022-01-03,Monday,[0],INVERNO,NaN
4785,2022-01-02,Sunday,[0],INVERNO,NaN


In [154]:
records_to_insert = records_to_insert.dropna(thresh=2)

In [155]:
records_to_insert

,data_aaaammdd,dia_semana,feriado,estacao,semana_do_mes
0,2009-12-31,Thursday,[0],INVERNO,NaN
1,2009-12-30,Wednesday,[0],INVERNO,NaN
2,2009-12-29,Tuesday,[0],INVERNO,NaN
3,2009-12-28,Monday,[0],INVERNO,NaN
4,2009-12-27,Sunday,[0],INVERNO,NaN
...,...,...,...,...,...
4782,2022-01-05,Wednesday,[0],INVERNO,NaN
4783,2022-01-04,Tuesday,[0],INVERNO,NaN
4784,2022-01-03,Monday,[0],INVERNO,NaN
4785,2022-01-02,Sunday,[0],INVERNO,NaN


In [ ]:
records_to_insert

In [ ]:
import sqlalchemy as db
class bd():

    def __init__(self):
        print('Criando conexão com o banco de dados')
    
    def conectado(self):
        db_server = 'Ironhack'
        user = 'postgres'
        password = senha_pessoal
        ip = '127.0.0.1:5432'
        db_name = 'postgres'
        #schema = 'db_projeto_cinema'
        engine = db.create_engine(f'{db_server}://{user}:{password}@{ip}/{db_name}')
        conn = engine.connect()
        return conn

In [796]:
db_server = 'postgresql'
user = 'postgres'
password = senha_pessoal
ip = '127.0.0.1'
db_name = 'postgres'
#schema = 'db_projeto_cinema'
print(f'{db_server}://{user}:{password}@{ip}/{db_name}')
engine = db.create_engine(f'{db_server}://{user}:{password}@{ip}/{db_name}')
conn = engine.connect()

NameError: name 'senha_pessoal' is not defined

In [157]:
records_to_insert.to_sql(name='tb_data', con=conn, if_exists='append', index=False, schema ='db_projeto_cinema' )

###  Segundo Webscrapping: Listagem de filme e bilheteria por dia
Neste segundo web scraping obteremos:
1) Os filmes, seu link, ranking no dia e estudio

2) Bilheteria do filme e numero de salas

In [15]:
df_data = df_data.drop_duplicates(subset = ['data_aaaammdd_link'])
df_data = df_data.dropna(thresh=1)
lista_link_dia_a_dia = df_data['data_aaaammdd_link']

print(lista_link_dia_a_dia)

0       2009-12-31
1       2009-12-30
2       2009-12-29
3       2009-12-28
4       2009-12-27
           ...    
4782    2022-01-05
4783    2022-01-04
4784    2022-01-03
4785    2022-01-02
4786    2022-01-01
Name: data_aaaammdd_link, Length: 4787, dtype: object


In [184]:
 # verificar como parametrizar esta parte

In [254]:
lista_release_id_dia = []
lista_link_release_id_dia = []
qual_dia = []
linha = 0

for table_row in lista_link_dia_a_dia:
    url_do_dia = f'https://www.boxofficemojo.com/date/{table_row}'
    response = requests.get(url_do_dia)
    html=response.content
    time.sleep(1)
    soup = BeautifulSoup(html)
    #print(soup.prettify())
    table_raw = soup.find('div', attrs={'id': 'table'})
    #print(table_raw.prettify())
    table_raw1 = table_raw.find('div', attrs={'class': 'a-section imdb-scroll-table-inner'})
    #print(table_raw1.prettify())
    table_rows = table_raw1.find_all('tr')
    #print(table_rows[1].prettify())
    for first_row in table_rows:
        lista_de_filmes = [elemento.text.strip() for elemento in first_row.find_all('td')]
        #print (lista_de_filmes)
        lista_release_id_dia.append(lista_de_filmes)
        qual_dia.append(table_row)
    
    print (table_row)

2009-12-31
2009-12-30
2009-12-29
2009-12-28
2009-12-27
2009-12-26
2009-12-25
2009-12-24
2009-12-23
2009-12-22
2009-12-21
2009-12-20
2009-12-19
2009-12-18
2009-12-17
2009-12-16
2009-12-15
2009-12-14
2009-12-13
2009-12-12
2009-12-11
2009-12-10
2009-12-09
2009-12-08
2009-12-07
2009-12-06
2009-12-05
2009-12-04
2009-12-03
2009-12-02
2009-12-01
2009-11-30
2009-11-29
2009-11-28
2009-11-27
2009-11-26
2009-11-25
2009-11-24
2009-11-23
2009-11-22
2009-11-21
2009-11-20
2009-11-19
2009-11-18
2009-11-17
2009-11-16
2009-11-15
2009-11-14
2009-11-13
2009-11-12
2009-11-11
2009-11-10
2009-11-09
2009-11-08
2009-11-07
2009-11-06
2009-11-05
2009-11-04
2009-11-03
2009-11-02
2009-11-01
2009-10-31
2009-10-30
2009-10-29
2009-10-28
2009-10-27
2009-10-26
2009-10-25
2009-10-24
2009-10-23
2009-10-22
2009-10-21
2009-10-20
2009-10-19
2009-10-18
2009-10-17
2009-10-16
2009-10-15
2009-10-14
2009-10-13
2009-10-12
2009-10-11
2009-10-10
2009-10-09
2009-10-08
2009-10-07
2009-10-06
2009-10-05
2009-10-04
2009-10-03
2009-10-02

2011-12-16
2011-12-15
2011-12-14
2011-12-13
2011-12-12
2011-12-11
2011-12-10
2011-12-09
2011-12-08
2011-12-07
2011-12-06
2011-12-05
2011-12-04
2011-12-03
2011-12-02
2011-12-01
2011-11-30
2011-11-29
2011-11-28
2011-11-27
2011-11-26
2011-11-25
2011-11-24
2011-11-23
2011-11-22
2011-11-21
2011-11-20
2011-11-19
2011-11-18
2011-11-17
2011-11-16
2011-11-15
2011-11-14
2011-11-13
2011-11-12
2011-11-11
2011-11-10
2011-11-09
2011-11-08
2011-11-07
2011-11-06
2011-11-05
2011-11-04
2011-11-03
2011-11-02
2011-11-01
2011-10-31
2011-10-30
2011-10-29
2011-10-28
2011-10-27
2011-10-26
2011-10-25
2011-10-24
2011-10-23
2011-10-22
2011-10-21
2011-10-20
2011-10-19
2011-10-18
2011-10-17
2011-10-16
2011-10-15
2011-10-14
2011-10-13
2011-10-12
2011-10-11
2011-10-10
2011-10-09
2011-10-08
2011-10-07
2011-10-06
2011-10-05
2011-10-04
2011-10-03
2011-10-02
2011-10-01
2011-09-30
2011-09-29
2011-09-28
2011-09-27
2011-09-26
2011-09-25
2011-09-24
2011-09-23
2011-09-22
2011-09-21
2011-09-20
2011-09-19
2011-09-18
2011-09-17

2013-12-02
2013-12-01
2013-11-30
2013-11-29
2013-11-28
2013-11-27
2013-11-26
2013-11-25
2013-11-24
2013-11-23
2013-11-22
2013-11-21
2013-11-20
2013-11-19
2013-11-18
2013-11-17
2013-11-16
2013-11-15
2013-11-14
2013-11-13
2013-11-12
2013-11-11
2013-11-10
2013-11-09
2013-11-08
2013-11-07
2013-11-06
2013-11-05
2013-11-04
2013-11-03
2013-11-02
2013-11-01
2013-10-31
2013-10-30
2013-10-29
2013-10-28
2013-10-27
2013-10-26
2013-10-25
2013-10-24
2013-10-23
2013-10-22
2013-10-21
2013-10-20
2013-10-19
2013-10-18
2013-10-17
2013-10-16
2013-10-15
2013-10-14
2013-10-13
2013-10-12
2013-10-11
2013-10-10
2013-10-09
2013-10-08
2013-10-07
2013-10-06
2013-10-05
2013-10-04
2013-10-03
2013-10-02
2013-10-01
2013-09-30
2013-09-29
2013-09-28
2013-09-27
2013-09-26
2013-09-25
2013-09-24
2013-09-23
2013-09-22
2013-09-21
2013-09-20
2013-09-19
2013-09-18
2013-09-17
2013-09-16
2013-09-15
2013-09-14
2013-09-13
2013-09-12
2013-09-11
2013-09-10
2013-09-09
2013-09-08
2013-09-07
2013-09-06
2013-09-05
2013-09-04
2013-09-03

2015-11-17
2015-11-16
2015-11-15
2015-11-14
2015-11-13
2015-11-12
2015-11-11
2015-11-10
2015-11-09
2015-11-08
2015-11-07
2015-11-06
2015-11-05
2015-11-04
2015-11-03
2015-11-02
2015-11-01
2015-10-31
2015-10-30
2015-10-29
2015-10-28
2015-10-27
2015-10-26
2015-10-25
2015-10-24
2015-10-23
2015-10-22
2015-10-21
2015-10-20
2015-10-19
2015-10-18
2015-10-17
2015-10-16
2015-10-15
2015-10-14
2015-10-13
2015-10-12
2015-10-11
2015-10-10
2015-10-09
2015-10-08
2015-10-07
2015-10-06
2015-10-05
2015-10-04
2015-10-03
2015-10-02
2015-10-01
2015-09-30
2015-09-29
2015-09-28
2015-09-27
2015-09-26
2015-09-25
2015-09-24
2015-09-23
2015-09-22
2015-09-21
2015-09-20
2015-09-19
2015-09-18
2015-09-17
2015-09-16
2015-09-15
2015-09-14
2015-09-13
2015-09-12
2015-09-11
2015-09-10
2015-09-09
2015-09-08
2015-09-07
2015-09-06
2015-09-05
2015-09-04
2015-09-03
2015-09-02
2015-09-01
2015-08-31
2015-08-30
2015-08-29
2015-08-28
2015-08-27
2015-08-26
2015-08-25
2015-08-24
2015-08-23
2015-08-22
2015-08-21
2015-08-20
2015-08-19

2017-11-03
2017-11-02
2017-11-01
2017-10-31
2017-10-30
2017-10-29
2017-10-28
2017-10-27
2017-10-26
2017-10-25
2017-10-24
2017-10-23
2017-10-22
2017-10-21
2017-10-20
2017-10-19
2017-10-18
2017-10-17
2017-10-16
2017-10-15
2017-10-14
2017-10-13
2017-10-12
2017-10-11
2017-10-10
2017-10-09
2017-10-08
2017-10-07
2017-10-06
2017-10-05
2017-10-04
2017-10-03
2017-10-02
2017-10-01
2017-09-30
2017-09-29
2017-09-28
2017-09-27
2017-09-26
2017-09-25
2017-09-24
2017-09-23
2017-09-22
2017-09-21
2017-09-20
2017-09-19
2017-09-18
2017-09-17
2017-09-16
2017-09-15
2017-09-14
2017-09-13
2017-09-12
2017-09-11
2017-09-10
2017-09-09
2017-09-08
2017-09-07
2017-09-06
2017-09-05
2017-09-04
2017-09-03
2017-09-02
2017-09-01
2017-08-31
2017-08-30
2017-08-29
2017-08-28
2017-08-27
2017-08-26
2017-08-25
2017-08-24
2017-08-23
2017-08-22
2017-08-21
2017-08-20
2017-08-19
2017-08-18
2017-08-17
2017-08-16
2017-08-15
2017-08-14
2017-08-13
2017-08-12
2017-08-11
2017-08-10
2017-08-09
2017-08-08
2017-08-07
2017-08-06
2017-08-05

2019-10-19
2019-10-18
2019-10-17
2019-10-16
2019-10-15
2019-10-14
2019-10-13
2019-10-12
2019-10-11
2019-10-10
2019-10-09
2019-10-08
2019-10-07
2019-10-06
2019-10-05
2019-10-04
2019-10-03
2019-10-02
2019-10-01
2019-09-30
2019-09-29
2019-09-28
2019-09-27
2019-09-26
2019-09-25
2019-09-24
2019-09-23
2019-09-22
2019-09-21
2019-09-20
2019-09-19
2019-09-18
2019-09-17
2019-09-16
2019-09-15
2019-09-14
2019-09-13
2019-09-12
2019-09-11
2019-09-10
2019-09-09
2019-09-08
2019-09-07
2019-09-06
2019-09-05
2019-09-04
2019-09-03
2019-09-02
2019-09-01
2019-08-31
2019-08-30
2019-08-29
2019-08-28
2019-08-27
2019-08-26
2019-08-25
2019-08-24
2019-08-23
2019-08-22
2019-08-21
2019-08-20
2019-08-19
2019-08-18
2019-08-17
2019-08-16
2019-08-15
2019-08-14
2019-08-13
2019-08-12
2019-08-11
2019-08-10
2019-08-09
2019-08-08
2019-08-07
2019-08-06
2019-08-05
2019-08-04
2019-08-03
2019-08-02
2019-08-01
2019-07-31
2019-07-30
2019-07-29
2019-07-28
2019-07-27
2019-07-26
2019-07-25
2019-07-24
2019-07-23
2019-07-22
2019-07-21

2021-09-29
2021-09-28
2021-09-27
2021-09-26
2021-09-25
2021-09-24
2021-09-23
2021-09-22
2021-09-21
2021-09-20
2021-09-19
2021-09-18
2021-09-17
2021-09-16
2021-09-15
2021-09-14
2021-09-13
2021-09-12
2021-09-11
2021-09-10
2021-09-09
2021-09-08
2021-09-07
2021-09-06
2021-09-05
2021-09-04
2021-09-03
2021-09-02
2021-09-01
2021-08-31
2021-08-30
2021-08-29
2021-08-28
2021-08-27
2021-08-26
2021-08-25
2021-08-24
2021-08-23
2021-08-22
2021-08-21
2021-08-20
2021-08-19
2021-08-18
2021-08-17
2021-08-16
2021-08-15
2021-08-14
2021-08-13
2021-08-12
2021-08-11
2021-08-10
2021-08-09
2021-08-08
2021-08-07
2021-08-06
2021-08-05
2021-08-04
2021-08-03
2021-08-02
2021-08-01
2021-07-31
2021-07-30
2021-07-29
2021-07-28
2021-07-27
2021-07-26
2021-07-25
2021-07-24
2021-07-23
2021-07-22
2021-07-21
2021-07-20
2021-07-19
2021-07-18
2021-07-17
2021-07-16
2021-07-15
2021-07-14
2021-07-13
2021-07-12
2021-07-11
2021-07-10
2021-07-09
2021-07-08
2021-07-07
2021-07-06
2021-07-05
2021-07-04
2021-07-03
2021-07-02
2021-07-01

In [255]:
    df_link_por_dia = pd.DataFrame(lista_release_id_dia,
            columns = ['td', 'yd', 'release', 'daily','perc_yd','perc_lw','theater','avg','todate','days', 'produtora','vazio1','vazio2'])

In [248]:
df_link_por_dia

,td,yd,release,daily,perc_yd,perc_lw,theater,avg,todate,days,produtora,vazio1,vazio2
0,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1,1,Avatar,"$14,738,136",-20.2%,+32.2%,"3,456","$4,264","$283,624,210",14,Twentieth Century Fox,false,false
2,2,2,Alvin and the Chipmunks: The Squeakquel,"$8,732,013",-25.9%,+10.4%,"3,700","$2,360","$120,727,282",9,Twentieth Century Fox,false,false
3,3,3,Sherlock Holmes,"$8,691,781",-10.6%,-,"3,626","$2,397","$102,102,956",7,Warner Bros.,false,false
4,4,4,It's Complicated,"$5,326,515",+21%,-,"2,897","$1,838","$40,375,875",7,Universal Pictures,false,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,40,40,The Men Who Stare at Goats,"$9,010",+3.8%,-49%,74,$121,"$31,970,324",55,Overture Films,false,false
85,41,41,The Box,"$4,876",+10%,-40.6%,50,$97,"$15,022,599",55,Warner Bros.,false,false
86,42,42,Amelia,"$3,467",+3.7%,-11.5%,24,$144,"$14,204,944",69,Fox Searchlight Pictures,false,false
87,43,43,The Maid,"$1,338",-0.2%,-41.6%,6,$223,"$485,076",76,Elephant Eye Films,false,false


In [256]:
lista_link_release_id_dia = []

for table_row in lista_link_dia_a_dia:
    url_do_dia = f'https://www.boxofficemojo.com/date/{table_row}' # verificar como parametrizar esta parte
    response = requests.get(url_do_dia)
    print(url_do_dia)
    html=response.content
    time.sleep(1)
    soup = BeautifulSoup(html)
    #print(soup.prettify())
    table_raw = soup.find('div', attrs={'id': 'table'})
    #print(table_raw.prettify())
    table_raw1 = table_raw.find('div', attrs={'class': 'a-section imdb-scroll-table-inner'})
    #print(table_raw1.prettify())
    table_rows = table_raw1.find_all('tr')
    #print(table_rows[1].prettify())
    
    for k in table_rows:
        link_desejado = extrair_link_top1(k)
        lista_link_release_id_dia.append(link_desejado)
        
    print (table_row)

https://www.boxofficemojo.com/date/2009-12-31
2009-12-31
https://www.boxofficemojo.com/date/2009-12-30
2009-12-30
https://www.boxofficemojo.com/date/2009-12-29
2009-12-29
https://www.boxofficemojo.com/date/2009-12-28
2009-12-28
https://www.boxofficemojo.com/date/2009-12-27
2009-12-27
https://www.boxofficemojo.com/date/2009-12-26
2009-12-26
https://www.boxofficemojo.com/date/2009-12-25
2009-12-25
https://www.boxofficemojo.com/date/2009-12-24
2009-12-24
https://www.boxofficemojo.com/date/2009-12-23
2009-12-23
https://www.boxofficemojo.com/date/2009-12-22
2009-12-22
https://www.boxofficemojo.com/date/2009-12-21
2009-12-21
https://www.boxofficemojo.com/date/2009-12-20
2009-12-20
https://www.boxofficemojo.com/date/2009-12-19
2009-12-19
https://www.boxofficemojo.com/date/2009-12-18
2009-12-18
https://www.boxofficemojo.com/date/2009-12-17
2009-12-17
https://www.boxofficemojo.com/date/2009-12-16
2009-12-16
https://www.boxofficemojo.com/date/2009-12-15
2009-12-15
https://www.boxofficemojo.com/d

2009-08-10
https://www.boxofficemojo.com/date/2009-08-09
2009-08-09
https://www.boxofficemojo.com/date/2009-08-08
2009-08-08
https://www.boxofficemojo.com/date/2009-08-07
2009-08-07
https://www.boxofficemojo.com/date/2009-08-06
2009-08-06
https://www.boxofficemojo.com/date/2009-08-05
2009-08-05
https://www.boxofficemojo.com/date/2009-08-04
2009-08-04
https://www.boxofficemojo.com/date/2009-08-03
2009-08-03
https://www.boxofficemojo.com/date/2009-08-02
2009-08-02
https://www.boxofficemojo.com/date/2009-08-01
2009-08-01
https://www.boxofficemojo.com/date/2009-07-31
2009-07-31
https://www.boxofficemojo.com/date/2009-07-30
2009-07-30
https://www.boxofficemojo.com/date/2009-07-29
2009-07-29
https://www.boxofficemojo.com/date/2009-07-28
2009-07-28
https://www.boxofficemojo.com/date/2009-07-27
2009-07-27
https://www.boxofficemojo.com/date/2009-07-26
2009-07-26
https://www.boxofficemojo.com/date/2009-07-25
2009-07-25
https://www.boxofficemojo.com/date/2009-07-24
2009-07-24
https://www.boxoffic

2009-03-19
https://www.boxofficemojo.com/date/2009-03-18
2009-03-18
https://www.boxofficemojo.com/date/2009-03-17
2009-03-17
https://www.boxofficemojo.com/date/2009-03-16
2009-03-16
https://www.boxofficemojo.com/date/2009-03-15
2009-03-15
https://www.boxofficemojo.com/date/2009-03-14
2009-03-14
https://www.boxofficemojo.com/date/2009-03-13
2009-03-13
https://www.boxofficemojo.com/date/2009-03-12
2009-03-12
https://www.boxofficemojo.com/date/2009-03-11
2009-03-11
https://www.boxofficemojo.com/date/2009-03-10
2009-03-10
https://www.boxofficemojo.com/date/2009-03-09
2009-03-09
https://www.boxofficemojo.com/date/2009-03-08
2009-03-08
https://www.boxofficemojo.com/date/2009-03-07
2009-03-07
https://www.boxofficemojo.com/date/2009-03-06
2009-03-06
https://www.boxofficemojo.com/date/2009-03-05
2009-03-05
https://www.boxofficemojo.com/date/2009-03-04
2009-03-04
https://www.boxofficemojo.com/date/2009-03-03
2009-03-03
https://www.boxofficemojo.com/date/2009-03-02
2009-03-02
https://www.boxoffic

2010-10-26
https://www.boxofficemojo.com/date/2010-10-25
2010-10-25
https://www.boxofficemojo.com/date/2010-10-24
2010-10-24
https://www.boxofficemojo.com/date/2010-10-23
2010-10-23
https://www.boxofficemojo.com/date/2010-10-22
2010-10-22
https://www.boxofficemojo.com/date/2010-10-21
2010-10-21
https://www.boxofficemojo.com/date/2010-10-20
2010-10-20
https://www.boxofficemojo.com/date/2010-10-19
2010-10-19
https://www.boxofficemojo.com/date/2010-10-18
2010-10-18
https://www.boxofficemojo.com/date/2010-10-17
2010-10-17
https://www.boxofficemojo.com/date/2010-10-16
2010-10-16
https://www.boxofficemojo.com/date/2010-10-15
2010-10-15
https://www.boxofficemojo.com/date/2010-10-14
2010-10-14
https://www.boxofficemojo.com/date/2010-10-13
2010-10-13
https://www.boxofficemojo.com/date/2010-10-12
2010-10-12
https://www.boxofficemojo.com/date/2010-10-11
2010-10-11
https://www.boxofficemojo.com/date/2010-10-10
2010-10-10
https://www.boxofficemojo.com/date/2010-10-09
2010-10-09
https://www.boxoffic

2010-06-04
https://www.boxofficemojo.com/date/2010-06-03
2010-06-03
https://www.boxofficemojo.com/date/2010-06-02
2010-06-02
https://www.boxofficemojo.com/date/2010-06-01
2010-06-01
https://www.boxofficemojo.com/date/2010-05-31
2010-05-31
https://www.boxofficemojo.com/date/2010-05-30
2010-05-30
https://www.boxofficemojo.com/date/2010-05-29
2010-05-29
https://www.boxofficemojo.com/date/2010-05-28
2010-05-28
https://www.boxofficemojo.com/date/2010-05-27
2010-05-27
https://www.boxofficemojo.com/date/2010-05-26
2010-05-26
https://www.boxofficemojo.com/date/2010-05-25
2010-05-25
https://www.boxofficemojo.com/date/2010-05-24
2010-05-24
https://www.boxofficemojo.com/date/2010-05-23
2010-05-23
https://www.boxofficemojo.com/date/2010-05-22
2010-05-22
https://www.boxofficemojo.com/date/2010-05-21
2010-05-21
https://www.boxofficemojo.com/date/2010-05-20
2010-05-20
https://www.boxofficemojo.com/date/2010-05-19
2010-05-19
https://www.boxofficemojo.com/date/2010-05-18
2010-05-18
https://www.boxoffic

...

2020-07-26
https://www.boxofficemojo.com/date/2020-07-25
2020-07-25
https://www.boxofficemojo.com/date/2020-07-24
2020-07-24
https://www.boxofficemojo.com/date/2020-07-23
2020-07-23
https://www.boxofficemojo.com/date/2020-07-22
2020-07-22
https://www.boxofficemojo.com/date/2020-07-21
2020-07-21
https://www.boxofficemojo.com/date/2020-07-20
2020-07-20
https://www.boxofficemojo.com/date/2020-07-19
2020-07-19
https://www.boxofficemojo.com/date/2020-07-18
2020-07-18
https://www.boxofficemojo.com/date/2020-07-17
2020-07-17
https://www.boxofficemojo.com/date/2020-07-16
2020-07-16
https://www.boxofficemojo.com/date/2020-07-15
2020-07-15
https://www.boxofficemojo.com/date/2020-07-14
2020-07-14
https://www.boxofficemojo.com/date/2020-07-13
2020-07-13
https://www.boxofficemojo.com/date/2020-07-12
2020-07-12
https://www.boxofficemojo.com/date/2020-07-11
2020-07-11
https://www.boxofficemojo.com/date/2020-07-10
2020-07-10
https://www.boxofficemojo.com/date/2020-07-09
2020-07-09
https://www.boxoffic

2020-02-27
https://www.boxofficemojo.com/date/2020-02-26
2020-02-26
https://www.boxofficemojo.com/date/2020-02-25
2020-02-25
https://www.boxofficemojo.com/date/2020-02-24
2020-02-24
https://www.boxofficemojo.com/date/2020-02-23
2020-02-23
https://www.boxofficemojo.com/date/2020-02-22
2020-02-22
https://www.boxofficemojo.com/date/2020-02-21
2020-02-21
https://www.boxofficemojo.com/date/2020-02-20
2020-02-20
https://www.boxofficemojo.com/date/2020-02-19
2020-02-19
https://www.boxofficemojo.com/date/2020-02-18
2020-02-18
https://www.boxofficemojo.com/date/2020-02-17
2020-02-17
https://www.boxofficemojo.com/date/2020-02-16
2020-02-16
https://www.boxofficemojo.com/date/2020-02-15
2020-02-15
https://www.boxofficemojo.com/date/2020-02-14
2020-02-14
https://www.boxofficemojo.com/date/2020-02-13
2020-02-13
https://www.boxofficemojo.com/date/2020-02-12
2020-02-12
https://www.boxofficemojo.com/date/2020-02-11
2020-02-11
https://www.boxofficemojo.com/date/2020-02-10
2020-02-10
https://www.boxoffic

2021-10-06
https://www.boxofficemojo.com/date/2021-10-05
2021-10-05
https://www.boxofficemojo.com/date/2021-10-04
2021-10-04
https://www.boxofficemojo.com/date/2021-10-03
2021-10-03
https://www.boxofficemojo.com/date/2021-10-02
2021-10-02
https://www.boxofficemojo.com/date/2021-10-01
2021-10-01
https://www.boxofficemojo.com/date/2021-09-30
2021-09-30
https://www.boxofficemojo.com/date/2021-09-29
2021-09-29
https://www.boxofficemojo.com/date/2021-09-28
2021-09-28
https://www.boxofficemojo.com/date/2021-09-27
2021-09-27
https://www.boxofficemojo.com/date/2021-09-26
2021-09-26
https://www.boxofficemojo.com/date/2021-09-25
2021-09-25
https://www.boxofficemojo.com/date/2021-09-24
2021-09-24
https://www.boxofficemojo.com/date/2021-09-23
2021-09-23
https://www.boxofficemojo.com/date/2021-09-22
2021-09-22
https://www.boxofficemojo.com/date/2021-09-21
2021-09-21
https://www.boxofficemojo.com/date/2021-09-20
2021-09-20
https://www.boxofficemojo.com/date/2021-09-19
2021-09-19
https://www.boxoffic

2021-05-15
https://www.boxofficemojo.com/date/2021-05-14
2021-05-14
https://www.boxofficemojo.com/date/2021-05-13
2021-05-13
https://www.boxofficemojo.com/date/2021-05-12
2021-05-12
https://www.boxofficemojo.com/date/2021-05-11
2021-05-11
https://www.boxofficemojo.com/date/2021-05-10
2021-05-10
https://www.boxofficemojo.com/date/2021-05-09
2021-05-09
https://www.boxofficemojo.com/date/2021-05-08
2021-05-08
https://www.boxofficemojo.com/date/2021-05-07
2021-05-07
https://www.boxofficemojo.com/date/2021-05-06
2021-05-06
https://www.boxofficemojo.com/date/2021-05-05
2021-05-05
https://www.boxofficemojo.com/date/2021-05-04
2021-05-04
https://www.boxofficemojo.com/date/2021-05-03
2021-05-03
https://www.boxofficemojo.com/date/2021-05-02
2021-05-02
https://www.boxofficemojo.com/date/2021-05-01
2021-05-01
https://www.boxofficemojo.com/date/2021-04-30
2021-04-30
https://www.boxofficemojo.com/date/2021-04-29
2021-04-29
https://www.boxofficemojo.com/date/2021-04-28
2021-04-28
https://www.boxoffic

2022-02-05
https://www.boxofficemojo.com/date/2022-02-04
2022-02-04
https://www.boxofficemojo.com/date/2022-02-03
2022-02-03
https://www.boxofficemojo.com/date/2022-02-02
2022-02-02
https://www.boxofficemojo.com/date/2022-02-01
2022-02-01
https://www.boxofficemojo.com/date/2022-01-31
2022-01-31
https://www.boxofficemojo.com/date/2022-01-30
2022-01-30
https://www.boxofficemojo.com/date/2022-01-29
2022-01-29
https://www.boxofficemojo.com/date/2022-01-28
2022-01-28
https://www.boxofficemojo.com/date/2022-01-27
2022-01-27
https://www.boxofficemojo.com/date/2022-01-26
2022-01-26
https://www.boxofficemojo.com/date/2022-01-25
2022-01-25
https://www.boxofficemojo.com/date/2022-01-24
2022-01-24
https://www.boxofficemojo.com/date/2022-01-23
2022-01-23
https://www.boxofficemojo.com/date/2022-01-22
2022-01-22
https://www.boxofficemojo.com/date/2022-01-21
2022-01-21
https://www.boxofficemojo.com/date/2022-01-20
2022-01-20
https://www.boxofficemojo.com/date/2022-01-19
2022-01-19
https://www.boxoffic

In [259]:
lista_link_release_id_dia

[None,
 'rl876971521',
 'rl1816364545',
 'rl3597567489',
 'rl2942600705',
 'rl3796338177',
 'rl3346433537',
 'rl3916137985',
 'rl2624357889',
 'rl827426305',
 'rl1667728897',
 'rl3479471617',
 'rl2621539841',
 'rl2605417985',
 'rl1835435521',
 'rl1833010689',
 'rl1095337473',
 'rl2875950593',
 'rl4883969',
 'rl1953269249',
 'rl3847259649',
 'rl2909898241',
 'rl964396545',
 'rl1699579393',
 'rl3580397057',
 'rl223184385',
 'rl3024258561',
 'rl1583842817',
 'rl3378742785',
 'rl3948905985',
 'rl2474018305',
 'rl3947005441',
 'rl3158738433',
 'rl1382974977',
 'rl2355725825',
 'rl122062337',
 'rl392922625',
 'rl140674561',
 'rl3328280065',
 'rl2589099521',
 'rl1413711361',
 'rl2353432065',
 'rl2304804353',
 'rl3882976769',
 None,
 'rl876971521',
 'rl1816364545',
 'rl3597567489',
 'rl2942600705',
 'rl3916137985',
 'rl3796338177',
 'rl3346433537',
 'rl2624357889',
 'rl1667728897',
 'rl3479471617',
 'rl827426305',
 'rl2621539841',
 'rl2605417985',
 'rl1095337473',
 'rl1833010689',
 'rl28759505

In [305]:
df_link_por_dia['data_aaaammdd'] = qual_dia

ValueError: Length of values (224190) does not match length of index (219403)

In [306]:
df_link_por_dia

,td,yd,release,daily,perc_yd,perc_lw,theater,avg,todate,days,produtora,vazio1,vazio2,data_aaaammdd,release_link
1,1,1,Avatar,"$14,738,136",-20.2%,+32.2%,"3,456","$4,264","$283,624,210",14,Twentieth Century Fox,NaN,false,2009-12-31,rl876971521
2,2,2,Alvin and the Chipmunks: The Squeakquel,"$8,732,013",-25.9%,+10.4%,"3,700","$2,360","$120,727,282",9,Twentieth Century Fox,NaN,false,2009-12-31,rl1816364545
3,3,3,Sherlock Holmes,"$8,691,781",-10.6%,-,"3,626","$2,397","$102,102,956",7,Warner Bros.,NaN,false,2009-12-31,rl3597567489
4,4,4,It's Complicated,"$5,326,515",+21%,-,"2,897","$1,838","$40,375,875",7,Universal Pictures,NaN,false,2009-12-31,rl2942600705
5,5,6,The Blind Side,"$3,465,225",+13%,+123%,"2,766","$1,252","$196,563,318",42,Warner Bros.,NaN,false,2009-12-31,rl3796338177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224185,21,23,Resident Evil: Welcome to Raccoon City,"$7,919",+294%,+52.9%,42,$188,"$16,931,810",39,Screen Gems,NaN,false,2022-01-01,rl829063937
224186,22,21,King Richard,"$5,681",+36.4%,+18.2%,50,$113,"$14,726,910",44,Warner Bros.,NaN,false,2022-01-01,rl1191740929
224187,23,22,C'mon C'mon,"$4,895",+34%,-29.6%,26,$188,"$1,828,248",44,A24,NaN,true,2022-01-01,rl2048033537
224188,24,24,Flee,$520,+5.5%,+27.1%,4,$130,"$55,425",30,Neon,NaN,false,2022-01-01,rl270762753


In [260]:
df_link_por_dia['release_link'] = lista_link_release_id_dia

In [261]:
df_link_por_dia

,td,yd,release,daily,perc_yd,perc_lw,theater,avg,todate,days,produtora,vazio1,vazio2,data_aaaammdd,release_link
0,None,None,None,None,None,None,None,None,None,None,None,None,None,2009-12-31,None
1,1,1,Avatar,"$14,738,136",-20.2%,+32.2%,"3,456","$4,264","$283,624,210",14,Twentieth Century Fox,false,false,2009-12-31,rl876971521
2,2,2,Alvin and the Chipmunks: The Squeakquel,"$8,732,013",-25.9%,+10.4%,"3,700","$2,360","$120,727,282",9,Twentieth Century Fox,false,false,2009-12-31,rl1816364545
3,3,3,Sherlock Holmes,"$8,691,781",-10.6%,-,"3,626","$2,397","$102,102,956",7,Warner Bros.,false,false,2009-12-31,rl3597567489
4,4,4,It's Complicated,"$5,326,515",+21%,-,"2,897","$1,838","$40,375,875",7,Universal Pictures,false,false,2009-12-31,rl2942600705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224185,21,23,Resident Evil: Welcome to Raccoon City,"$7,919",+294%,+52.9%,42,$188,"$16,931,810",39,Screen Gems,false,false,2022-01-01,rl829063937
224186,22,21,King Richard,"$5,681",+36.4%,+18.2%,50,$113,"$14,726,910",44,Warner Bros.,false,false,2022-01-01,rl1191740929
224187,23,22,C'mon C'mon,"$4,895",+34%,-29.6%,26,$188,"$1,828,248",44,A24,false,true,2022-01-01,rl2048033537
224188,24,24,Flee,$520,+5.5%,+27.1%,4,$130,"$55,425",30,Neon,false,false,2022-01-01,rl270762753


In [262]:
df_link_por_dia = df_link_por_dia.drop_duplicates(subset = ['release_link','data_aaaammdd'])
df_link_por_dia = df_link_por_dia.dropna(thresh=4)

In [263]:
df_link_por_dia

,td,yd,release,daily,perc_yd,perc_lw,theater,avg,todate,days,produtora,vazio1,vazio2,data_aaaammdd,release_link
1,1,1,Avatar,"$14,738,136",-20.2%,+32.2%,"3,456","$4,264","$283,624,210",14,Twentieth Century Fox,false,false,2009-12-31,rl876971521
2,2,2,Alvin and the Chipmunks: The Squeakquel,"$8,732,013",-25.9%,+10.4%,"3,700","$2,360","$120,727,282",9,Twentieth Century Fox,false,false,2009-12-31,rl1816364545
3,3,3,Sherlock Holmes,"$8,691,781",-10.6%,-,"3,626","$2,397","$102,102,956",7,Warner Bros.,false,false,2009-12-31,rl3597567489
4,4,4,It's Complicated,"$5,326,515",+21%,-,"2,897","$1,838","$40,375,875",7,Universal Pictures,false,false,2009-12-31,rl2942600705
5,5,6,The Blind Side,"$3,465,225",+13%,+123%,"2,766","$1,252","$196,563,318",42,Warner Bros.,false,false,2009-12-31,rl3796338177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224185,21,23,Resident Evil: Welcome to Raccoon City,"$7,919",+294%,+52.9%,42,$188,"$16,931,810",39,Screen Gems,false,false,2022-01-01,rl829063937
224186,22,21,King Richard,"$5,681",+36.4%,+18.2%,50,$113,"$14,726,910",44,Warner Bros.,false,false,2022-01-01,rl1191740929
224187,23,22,C'mon C'mon,"$4,895",+34%,-29.6%,26,$188,"$1,828,248",44,A24,false,true,2022-01-01,rl2048033537
224188,24,24,Flee,$520,+5.5%,+27.1%,4,$130,"$55,425",30,Neon,false,false,2022-01-01,rl270762753


In [418]:
'''
TB_FATURAMENTO:

release_link VARCHAR(256) NOT NULL, -- 
data_aaaammdd TIMESTAMP NOT NULL, -- 
posicao_atual INT NOT NULL, -- 
posicao_anterior INT, --
salas_em_exibicao INT NOT NULL, --
faturamento_diario BIGINT NOT NULL, --
tag_analise VARCHAR(256) NOT NULL,
'''

records_to_faturamento = df_link_por_dia[['release_link','data_aaaammdd', 'td', 'yd', 'theater','daily','release']]
records_to_faturamento = records_to_faturamento.rename(columns={'td': 'posicao_atual'})
records_to_faturamento = records_to_faturamento.rename(columns={'yd': 'posicao_anterior'})
records_to_faturamento = records_to_faturamento.rename(columns={'theater': 'salas_em_exibicao'})
records_to_faturamento = records_to_faturamento.rename(columns={'yd': 'posicao_anterior'})
records_to_faturamento = records_to_faturamento.rename(columns={'daily': 'faturamento_diario'})
records_to_faturamento = records_to_faturamento.rename(columns={'release': 'tag_analise'})

records_to_faturamento


,release_link,data_aaaammdd,posicao_atual,posicao_anterior,salas_em_exibicao,faturamento_diario,tag_analise
1,rl876971521,2009-12-31,1,1,"3,456","$14,738,136",Avatar
2,rl1816364545,2009-12-31,2,2,"3,700","$8,732,013",Alvin and the Chipmunks: The Squeakquel
3,rl3597567489,2009-12-31,3,3,"3,626","$8,691,781",Sherlock Holmes
4,rl2942600705,2009-12-31,4,4,"2,897","$5,326,515",It's Complicated
5,rl3796338177,2009-12-31,5,6,"2,766","$3,465,225",The Blind Side
...,...,...,...,...,...,...,...
224185,rl829063937,2022-01-01,21,23,42,"$7,919",Resident Evil: Welcome to Raccoon City
224186,rl1191740929,2022-01-01,22,21,50,"$5,681",King Richard
224187,rl2048033537,2022-01-01,23,22,26,"$4,895",C'mon C'mon
224188,rl270762753,2022-01-01,24,24,4,$520,Flee


In [419]:
# Importing re package for using regular expressions

  
# Function to clean the names
def limpa_cifrao(valor):
    padrao = f'\D'
    novo_valor = re.sub(padrao,'',valor)
    return novo_valor

records_to_faturamento['faturamento_diario'][5]
print(limpa_cifrao(records_to_faturamento['faturamento_diario'][5]))

3465225


In [420]:
records_to_faturamento['faturamento_diario'] = records_to_faturamento['faturamento_diario'].apply(limpa_cifrao)
records_to_faturamento['salas_em_exibicao'] = records_to_faturamento['salas_em_exibicao'].apply(limpa_cifrao)

In [421]:
records_to_faturamento

,release_link,data_aaaammdd,posicao_atual,posicao_anterior,salas_em_exibicao,faturamento_diario,tag_analise
1,rl876971521,2009-12-31,1,1,3456,14738136,Avatar
2,rl1816364545,2009-12-31,2,2,3700,8732013,Alvin and the Chipmunks: The Squeakquel
3,rl3597567489,2009-12-31,3,3,3626,8691781,Sherlock Holmes
4,rl2942600705,2009-12-31,4,4,2897,5326515,It's Complicated
5,rl3796338177,2009-12-31,5,6,2766,3465225,The Blind Side
...,...,...,...,...,...,...,...
224185,rl829063937,2022-01-01,21,23,42,7919,Resident Evil: Welcome to Raccoon City
224186,rl1191740929,2022-01-01,22,21,50,5681,King Richard
224187,rl2048033537,2022-01-01,23,22,26,4895,C'mon C'mon
224188,rl270762753,2022-01-01,24,24,4,520,Flee


In [422]:
records_to_faturamento['faturamento_diario'] = records_to_faturamento['faturamento_diario'].apply(limpa_cifrao)

In [423]:
vazio1 = []
for i in df_link_por_dia['vazio1']:
    vazio1.append(np.NaN)
    
df_link_por_dia['vazio1'] = vazio1

In [412]:
'''
postgres.db_projeto_cinema.TB_FILMES
release_link VARCHAR(256) NOT NULL, -- chave do filme
nome_release VARCHAR(256) NOT NULL, -- nome do filme
imdb_link VARCHAR(256) , -- chave para imdb
entrada_em_cartaz TIMESTAMP , -- referencia
saida_de_cartaz TIMESTAMP, -- referencia
orcamento BIGINT , 
produtora_link VARCHAR(256) , 
salas_exibidas BIGINT ,
expectadores BIGINT ,
mpaa VARCHAR(256) ,
duracao_do_filme TIMESTAMP ,
id_genero VARCHAR(256) ,
watch_time_minutes INT , -- tag no boxofficemojo
link_poster VARCHAR(256) 
'''

filmes = df_link_por_dia[['release_link','release','vazio1','vazio1','vazio1','vazio1','produtora','vazio1','vazio1','vazio1','vazio1','vazio1','vazio1','vazio1']]
filmes.columns = ['release_link', 'nome_release','imdb_link','entrada_em_cartaz','saida_de_cartaz','orcamento','produtora_link','salas_exibidas','expectadores','mpaa','duracao_do_filme','id_genero','watch_time_minutes','link_poster']

In [407]:
selecao2 = filmes['release_link']== 'rl876971521'
filmes = filmes[selecao2]

In [413]:
filmes

,release_link,nome_release,imdb_link,entrada_em_cartaz,saida_de_cartaz,orcamento,produtora_link,salas_exibidas,expectadores,mpaa,duracao_do_filme,id_genero,watch_time_minutes,link_poster
1,rl876971521,Avatar,NaN,NaN,NaN,NaN,Twentieth Century Fox,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rl1816364545,Alvin and the Chipmunks: The Squeakquel,NaN,NaN,NaN,NaN,Twentieth Century Fox,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rl3597567489,Sherlock Holmes,NaN,NaN,NaN,NaN,Warner Bros.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rl2942600705,It's Complicated,NaN,NaN,NaN,NaN,Universal Pictures,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,rl3796338177,The Blind Side,NaN,NaN,NaN,NaN,Warner Bros.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224185,rl829063937,Resident Evil: Welcome to Raccoon City,NaN,NaN,NaN,NaN,Screen Gems,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224186,rl1191740929,King Richard,NaN,NaN,NaN,NaN,Warner Bros.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224187,rl2048033537,C'mon C'mon,NaN,NaN,NaN,NaN,A24,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224188,rl270762753,Flee,NaN,NaN,NaN,NaN,Neon,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [414]:
filmes = filmes.drop_duplicates(subset = ['release_link'])

In [415]:
filmes

,release_link,nome_release,imdb_link,entrada_em_cartaz,saida_de_cartaz,orcamento,produtora_link,salas_exibidas,expectadores,mpaa,duracao_do_filme,id_genero,watch_time_minutes,link_poster
1,rl876971521,Avatar,NaN,NaN,NaN,NaN,Twentieth Century Fox,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rl1816364545,Alvin and the Chipmunks: The Squeakquel,NaN,NaN,NaN,NaN,Twentieth Century Fox,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rl3597567489,Sherlock Holmes,NaN,NaN,NaN,NaN,Warner Bros.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rl2942600705,It's Complicated,NaN,NaN,NaN,NaN,Universal Pictures,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,rl3796338177,The Blind Side,NaN,NaN,NaN,NaN,Warner Bros.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223545,rl2267710209,Shortbus2022 Re-release (Restoration),NaN,NaN,NaN,NaN,Oscilloscope,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223783,rl2718728961,Parallel Mothers,NaN,NaN,NaN,NaN,Sony Pictures Classics,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223787,rl1527939841,Who We Are: A Chronicle of Racism in America,NaN,NaN,NaN,NaN,Sony Pictures Entertainment (SPE),NaN,NaN,NaN,NaN,NaN,NaN,NaN
223790,rl3358294785,Alter Ego,NaN,NaN,NaN,NaN,Indican Pictures,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [347]:
filmes['release_link'].count()

4486

In [381]:
filmes.loc[[1]]

,release_link,nome_release,imdb_link,entrada_em_cartaz,saida_de_cartaz,orcamento,produtora_link,salas_exibidas,expectadores,mpaa,duracao_do_filme,id_genero,watch_time_minutes,link_poster
1,rl876971521,Avatar,NaN,NaN,NaN,NaN,Twentieth Century Fox,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [466]:
filmes.to_sql(name='tb_filmes', con=conn, if_exists='append', index=False, schema ='db_projeto_cinema' )

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "tb_filmes_pkey"
DETAIL:  Key (release_link)=(rl876971521) already exists.

[SQL: INSERT INTO db_projeto_cinema.tb_filmes (release_link, nome_release, imdb_link, entrada_em_cartaz, saida_de_cartaz, orcamento, produtora_link, salas_exibidas, expectadores, mpaa, duracao_do_filme, id_genero, watch_time_minutes, link_poster) VALUES (%(release_link)s, %(nome_release)s, %(imdb_link)s, %(entrada_em_cartaz)s, %(saida_de_cartaz)s, %(orcamento)s, %(produtora_link)s, %(salas_exibidas)s, %(expectadores)s, %(mpaa)s, %(duracao_do_filme)s, %(id_genero)s, %(watch_time_minutes)s, %(link_poster)s)]
[parameters: ({'release_link': 'rl876971521', 'nome_release': 'Avatar', 'imdb_link': None, 'entrada_em_cartaz': None, 'saida_de_cartaz': None, 'orcamento': None, 'produtora_link': 'Twentieth Century Fox', 'salas_exibidas': None, 'expectadores': None, 'mpaa': None, 'duracao_do_filme': None, 'id_genero': None, 'watc

In [467]:
from sqlalchemy.exc import IntegrityError

In [470]:
df = filmes.loc[[0+1]]

In [471]:
df

,release_link,nome_release,imdb_link,entrada_em_cartaz,saida_de_cartaz,orcamento,produtora_link,salas_exibidas,expectadores,mpaa,duracao_do_filme,id_genero,watch_time_minutes,link_poster
1,rl876971521,Avatar,NaN,NaN,NaN,NaN,Twentieth Century Fox,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [476]:
for i in range(filmes['release_link'].count()):
    try:
        df = filmes.loc[[i+1]]
        print(df)
        df.to_sql(name='tb_filmes', con=conn, if_exists='append', index= True, index_label = 'release_link', schema ='db_projeto_cinema' ) 
        #SQL: nao usar o to_sql, usar sqlalchemy
    except IntegrityError:
        pass 
    
    except ValueError:
        pass

    except KeyError:
        pass

  release_link nome_release  imdb_link  entrada_em_cartaz  saida_de_cartaz  \
1  rl876971521       Avatar        NaN                NaN              NaN   

   orcamento         produtora_link  salas_exibidas  expectadores  mpaa  \
1        NaN  Twentieth Century Fox             NaN           NaN   NaN   

   duracao_do_filme  id_genero  watch_time_minutes  link_poster  
1               NaN        NaN                 NaN          NaN  
   release_link                             nome_release  imdb_link  \
2  rl1816364545  Alvin and the Chipmunks: The Squeakquel        NaN   

   entrada_em_cartaz  saida_de_cartaz  orcamento         produtora_link  \
2                NaN              NaN        NaN  Twentieth Century Fox   

   salas_exibidas  expectadores  mpaa  duracao_do_filme  id_genero  \
2             NaN           NaN   NaN               NaN        NaN   

   watch_time_minutes  link_poster  
2                 NaN          NaN  
   release_link     nome_release  imdb_link  entrad

    release_link       nome_release  imdb_link  entrada_em_cartaz  \
20  rl3847259649  Fantastic Mr. Fox        NaN                NaN   

    saida_de_cartaz  orcamento         produtora_link  salas_exibidas  \
20              NaN        NaN  Twentieth Century Fox             NaN   

    expectadores  mpaa  duracao_do_filme  id_genero  watch_time_minutes  \
20           NaN   NaN               NaN        NaN                 NaN   

    link_poster  
20          NaN  
    release_link  nome_release  imdb_link  entrada_em_cartaz  saida_de_cartaz  \
21  rl2909898241  A Single Man        NaN                NaN              NaN   

    orcamento         produtora_link  salas_exibidas  expectadores  mpaa  \
21        NaN  The Weinstein Company             NaN           NaN   NaN   

    duracao_do_filme  id_genero  watch_time_minutes  link_poster  
21               NaN        NaN                 NaN          NaN  
   release_link               nome_release  imdb_link  entrada_em_cartaz  \
2

    release_link nome_release  imdb_link  entrada_em_cartaz  saida_de_cartaz  \
42  rl2304804353     The Maid        NaN                NaN              NaN   

    orcamento      produtora_link  salas_exibidas  expectadores  mpaa  \
42        NaN  Elephant Eye Films             NaN           NaN   NaN   

    duracao_do_filme  id_genero  watch_time_minutes  link_poster  
42               NaN        NaN                 NaN          NaN  
    release_link         nome_release  imdb_link  entrada_em_cartaz  \
43  rl3882976769  The Slammin' Salmon        NaN                NaN   

    saida_de_cartaz  orcamento    produtora_link  salas_exibidas  \
43              NaN        NaN  Anchor Bay Films             NaN   

    expectadores  mpaa  duracao_do_filme  id_genero  watch_time_minutes  \
43           NaN   NaN               NaN        NaN                 NaN   

    link_poster  
43          NaN  
    release_link        nome_release  imdb_link  entrada_em_cartaz  \
61  rl4269442561  The

    release_link nome_release  imdb_link  entrada_em_cartaz  saida_de_cartaz  \
883  rl847087105   Zombieland        NaN                NaN              NaN   

     orcamento                     produtora_link  salas_exibidas  \
883        NaN  Sony Pictures Entertainment (SPE)             NaN   

     expectadores  mpaa  duracao_do_filme  id_genero  watch_time_minutes  \
883           NaN   NaN               NaN        NaN                 NaN   

     link_poster  
883          NaN  
    release_link                    nome_release  imdb_link  \
909  rl877102593  Bicycle Thieves2008 Re-release        NaN   

     entrada_em_cartaz  saida_de_cartaz  orcamento produtora_link  \
909                NaN              NaN        NaN  Corinth Films   

     salas_exibidas  expectadores  mpaa  duracao_do_filme  id_genero  \
909             NaN           NaN   NaN               NaN        NaN   

     watch_time_minutes  link_poster  
909                 NaN          NaN  
     release_link no

     release_link nome_release  imdb_link  entrada_em_cartaz  saida_de_cartaz  \
2134   rl58099201     Twilight        NaN                NaN              NaN   

      orcamento        produtora_link  salas_exibidas  expectadores  mpaa  \
2134        NaN  Summit Entertainment             NaN           NaN   NaN   

      duracao_do_filme  id_genero  watch_time_minutes  link_poster  
2134               NaN        NaN                 NaN          NaN  
      release_link     nome_release  imdb_link  entrada_em_cartaz  \
2166  rl3831006721  The Hurt Locker        NaN                NaN   

      saida_de_cartaz  orcamento        produtora_link  salas_exibidas  \
2166              NaN        NaN  Summit Entertainment             NaN   

      expectadores  mpaa  duracao_do_filme  id_genero  watch_time_minutes  \
2166           NaN   NaN               NaN        NaN                 NaN   

      link_poster  
2166          NaN  
      release_link     nome_release  imdb_link  entrada_em_ca

      release_link nome_release  imdb_link  entrada_em_cartaz  \
3186  rl3914302977   District 9        NaN                NaN   

      saida_de_cartaz  orcamento    produtora_link  salas_exibidas  \
3186              NaN        NaN  TriStar Pictures             NaN   

      expectadores  mpaa  duracao_do_filme  id_genero  watch_time_minutes  \
3186           NaN   NaN               NaN        NaN                 NaN   

      link_poster  
3186          NaN  
      release_link     nome_release  imdb_link  entrada_em_cartaz  \
3211  rl1296270849  American Casino        NaN                NaN   

      saida_de_cartaz  orcamento  produtora_link  salas_exibidas  \
3211              NaN        NaN  Argot Pictures             NaN   

      expectadores  mpaa  duracao_do_filme  id_genero  watch_time_minutes  \
3211           NaN   NaN               NaN        NaN                 NaN   

      link_poster  
3211          NaN  
      release_link              nome_release  imdb_link  entra

In [436]:
def limpa_sem_posicao(tracinho):
    if tracinho =='':
        return '0'
    elif tracinho == '-': 
        return re.sub('-','0',tracinho)

In [432]:
def sala_em_branco (em_branco):
    if em_branco == np.NaN:
        return '0'
    elif em_branco == '':
        return '0'
    else:
        return em_branco

In [437]:
records_to_faturamento['posicao_anterior'] = records_to_faturamento['posicao_anterior'].apply(limpa_sem_posicao)

In [434]:
records_to_faturamento['salas_em_exibicao'] = records_to_faturamento['salas_em_exibicao'].apply(sala_em_branco)

In [438]:
records_to_faturamento.to_sql(name='tb_faturamento', con=conn, if_exists='append', index=False, schema ='db_projeto_cinema' )

In [309]:
selecao = records_to_faturamento['release_link']== 'rl1970046465'
records_to_faturamento[selecao]

,release_link,data_aaaammdd,posicao_atual,posicao_anterior,salas_em_exibicao,faturamento_diario,tag_analise
63108,rl1970046465,2012-05-24,40,None,10,1123,The Road
63160,rl1970046465,2012-05-23,42,None,10,863,The Road
63211,rl1970046465,2012-05-22,42,None,10,815,The Road
63262,rl1970046465,2012-05-21,42,None,10,804,The Road
63313,rl1970046465,2012-05-20,42,None,10,2033,The Road
63367,rl1970046465,2012-05-19,42,None,10,2440,The Road
63422,rl1970046465,2012-05-18,43,None,10,1651,The Road
63469,rl1970046465,2012-05-17,35,None,50,5115,The Road
63524,rl1970046465,2012-05-16,34,None,50,5503,The Road
63576,rl1970046465,2012-05-15,30,None,50,7920,The Road


In [310]:
selecao2 = filmes['release_link']== 'rl1970046465'
filmes[selecao2]

,release_link,nome_release,imdb_link,entrada_em_cartaz,saida_de_cartaz,orcamento,produtora_link,salas_exibidas,expectadores,mpaa,duracao_do_filme,id_genero,watch_time_minutes,link_poster


In [454]:
df_link_por_dia['produtora']

1         Twentieth Century Fox
2         Twentieth Century Fox
3                  Warner Bros.
4            Universal Pictures
5                  Warner Bros.
                  ...          
224185              Screen Gems
224186             Warner Bros.
224187                      A24
224188                     Neon
224189             CMC Pictures
Name: produtora, Length: 219403, dtype: object

In [500]:
'''
CREATE TABLE postgres.db_projeto_cinema.TB_PRODUTORAS

produtora_link VARCHAR(256) NOT NULL PRIMARY KEY, -- 
desc_produtora VARCHAR(256) NOT NULL
'''

produtora = pd.DataFrame(df_link_por_dia['produtora'])
produtora.columns = ['produtora_link']


In [501]:
produtora

,produtora_link
1,Twentieth Century Fox
2,Twentieth Century Fox
3,Warner Bros.
4,Universal Pictures
5,Warner Bros.
...,...
224185,Screen Gems
224186,Warner Bros.
224187,A24
224188,Neon


In [502]:
produtora['desc_produtora'] = produtora['produtora_link']

In [503]:
produtora

,produtora_link,desc_produtora
1,Twentieth Century Fox,Twentieth Century Fox
2,Twentieth Century Fox,Twentieth Century Fox
3,Warner Bros.,Warner Bros.
4,Universal Pictures,Universal Pictures
5,Warner Bros.,Warner Bros.
...,...,...
224185,Screen Gems,Screen Gems
224186,Warner Bros.,Warner Bros.
224187,A24,A24
224188,Neon,Neon


In [504]:
produtora = produtora.drop_duplicates()

In [505]:
produtora

,produtora_link,desc_produtora
1,Twentieth Century Fox,Twentieth Century Fox
3,Warner Bros.,Warner Bros.
4,Universal Pictures,Universal Pictures
6,Paramount Pictures,Paramount Pictures
7,Walt Disney Studios Motion Pictures,Walt Disney Studios Motion Pictures
...,...,...
221288,Flyhigh Cinemas,Flyhigh Cinemas
222483,54&O Productions,54&O Productions
222503,Entertainment Squad,Entertainment Squad
222550,Nova Vento Entertainment,Nova Vento Entertainment


In [509]:
produtora.to_sql(name='tb_produtoras', con=conn, if_exists='append', index=True, index_label = 'produtora_link', schema ='db_projeto_cinema' )

ValueError: duplicate name in index/columns: cannot insert produtora_link, already exists

###  Terceiro Webscrapping: Listagem de filme dos filmes do periodo
Neste terceiro web scraping obteremos:
1) O filme, seu link BoxOfficeMojo, seu link IMDb

2) Faturamento no Mercado Norte Americano, Internacional e Total

3) Distributor

4) Salas em exibicao

5) Budget

6) Data de lancamento

7) MPAA

8) Duracao do filme

9) Genres

10) Tempo em exibicao

11) Total de salas exibido

12) Link IMDbPro

13) Faturamento por dia em cartaz

A partir do webscraping anterior sera obtido aslgumas informacoes sobre cada filme
Exemplo de link da pagina anterior: https://www.boxofficemojo.com/release/rl2869659137

Como os filmes se repetem, sera necessario fazer um group by para evitar duplicidades:

In [527]:
# seleciona a lista de filmes para terceiro webscrapping:
lista_de_filmes = filmes ['release_link']

In [528]:
lista_de_filmes

1          rl876971521
2         rl1816364545
3         rl3597567489
4         rl2942600705
5         rl3796338177
              ...     
223545    rl2267710209
223783    rl2718728961
223787    rl1527939841
223790    rl3358294785
223894     rl689341185
Name: release_link, Length: 4486, dtype: object

In [ ]:
# Fazer o web scraping 3: dados dos filmes

In [533]:
lista_de_filmes

1          rl876971521
2         rl1816364545
3         rl3597567489
4         rl2942600705
5         rl3796338177
              ...     
223545    rl2267710209
223783    rl2718728961
223787    rl1527939841
223790    rl3358294785
223894     rl689341185
Name: release_link, Length: 4486, dtype: object

In [538]:
link_id_filme_boxofficemojo = lista_de_filmes.iloc[5]
link_id_filme_boxofficemojo

'rl3346433537'

In [539]:
'''

    url_do_dia = f'https://www.boxofficemojo.com/date/{table_row}' # verificar como parametrizar esta parte
    response = requests.get(url_do_dia)
    print(url_do_dia)
    html=response.content
    time.sleep(1)
    soup = BeautifulSoup(html)
    #print(soup.prettify())
    table_raw = soup.find('div', attrs={'id': 'table'})
    #print(table_raw.prettify())
    table_raw1 = table_raw.find('div', attrs={'class': 'a-section imdb-scroll-table-inner'})
    #print(table_raw1.prettify())
    table_rows = table_raw1.find_all('tr')
    #print(table_rows[1].prettify())
    
    for k in table_rows:
        link_desejado = extrair_link_top1(k)
        lista_link_release_id_dia.append(link_desejado)
        
    print (table_row)
    
    
'''

https://www.boxofficemojo.com/release/rl3346433537


In [618]:
link_id_filme_boxofficemojo

'rl3346433537'

In [628]:
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.implicitly_wait(0.5)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST chromedriver version for 93.0.4577 google-chrome
Driver [/Users/Lewincarol/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


In [664]:
driver.get("https://www.boxofficemojo.com/release/rl3346433537")
driver.implicitly_wait(3)

In [665]:
barras = driver.find_element(By.XPATH, "//div[@class='a-section a-spacing-none mojo-gutter mojo-summary-table']")

In [713]:
barras_direita = barras.find_element(By.XPATH, "//div[@class='a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile']")

In [714]:
print(barras_direita)

<selenium.webdriver.remote.webelement.WebElement (session="62a140de9f419f8a8401b671187bd70e", element="a7cc124f-113b-43df-8fc0-f03d680d2218")>


In [715]:
barras_direita_n1 = barras_direita.find_elements(By.XPATH, "//span[@class = 'money']")

In [719]:
a = [x.text for x in barras_direita_n1]
print(a)

['$83,823,381', '$83,019,358', '$166,842,739', '$1,181,450', '$25,000,000']


In [724]:
barras_direita_n2 = barras_direita.find_elements(By.XPATH, "//span[@class = 'a-link-normal']")
barras_direita_n3 = barras_direita.find_elements(By.XPATH, "//href")

In [725]:
print(barras_direita_n3)

[]


In [697]:
print(barras_direita_n3)

NameError: name 'barras_direita_n3' is not defined

In [778]:
nova_lista = [
'rl1551074049',
'rl3803873793',
'rl34178561',
'rl829063937',
'rl4077355777']

In [779]:
nova_lista

['rl1551074049', 'rl3803873793', 'rl34178561', 'rl829063937', 'rl4077355777']

In [658]:
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.implicitly_wait(0.5)

In [784]:
lista_valores = []
filmes_plug = []
#default eh lista_de_filmes
nova_lista


for release in nova_lista:

    url_do_filme = f'https://www.boxofficemojo.com/release/{release}' # verificar como parametrizar esta parteresponse = requests.get(url_do_dia)
    print(url_do_filme)
    driver.get(url_do_filme)
    driver.implicitly_wait(5)

    barras = driver.find_element(By.XPATH, "//div[@class='a-section a-spacing-none mojo-gutter mojo-summary-table']")

    barras_direita = barras.find_element(By.XPATH, "//div[@class='a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile']")

    barras_direita_n1 = barras_direita.find_elements(By.XPATH, "//span[@class = 'money']")

    valores_money = [x.text for x in barras_direita_n1]
    lista_valores.append(valores_money)
    filmes_plug.append(release)
    
    print(release)

https://www.boxofficemojo.com/release/rl1551074049
rl1551074049
https://www.boxofficemojo.com/release/rl3803873793
rl3803873793
https://www.boxofficemojo.com/release/rl34178561
rl34178561
https://www.boxofficemojo.com/release/rl829063937
rl829063937
https://www.boxofficemojo.com/release/rl4077355777
rl4077355777


In [787]:
df_valores = pd.DataFrame(lista_valores)
df_valores

,0,1,2,3
0,"$2,807,494","$53,193","$2,860,687","$1,042,171"
1,"$33,419","$241,255","$274,674","$12,850"
2,"$1,565,885","$118,366,848","$119,932,733","$42,587"
3,"$17,000,612","$21,600,000","$38,600,612","$5,319,280"
4,"$16,104","$16,104",None,None


In [792]:
try:
    df_valores = pd.DataFrame(lista_valores,columns=['US','INTL','TOTAL','ESPECTADORES','ORCAMENTO'])
    df_valores['release_link'] = filmes_plug
    
except ValueError:
    df_valores = pd.DataFrame(lista_valores,columns=['US','INTL','TOTAL','ESPECTADORES'])
    df_valores['ORCAMENTO'] = [np.NaN for i in filmes_plug]
    df_valores['release_link'] = filmes_plug

df_valores

,US,INTL,TOTAL,ESPECTADORES,ORCAMENTO,release_link
0,"$2,807,494","$53,193","$2,860,687","$1,042,171",NaN,rl1551074049
1,"$33,419","$241,255","$274,674","$12,850",NaN,rl3803873793
2,"$1,565,885","$118,366,848","$119,932,733","$42,587",NaN,rl34178561
3,"$17,000,612","$21,600,000","$38,600,612","$5,319,280",NaN,rl829063937
4,"$16,104","$16,104",None,None,NaN,rl4077355777


In [793]:
'''
CREATE TABLE postgres.db_projeto_cinema.TEMP_FILMES_DADOS
(
--sera utilizada para fazer update da tabela filmes
release_link VARCHAR(256) NOT NULL -- 
,fat_us BIGINT -- 
,fat_intl BIGINT -- 
,fat_total BIGINT --
,espectadores BIGINT--
,orcamento BIGINT --
);
'''
df_valores_organizada = df_valores[['release_link','US','INTL','TOTAL','ESPECTADORES','ORCAMENTO']]
df_valores_organizada.columns = ['release_link','fat_us','fat_intl','fat_total','espectadores','orcamento']

In [797]:
db_server = 'postgresql'
user = 'postgres'
password = senha_pessoal
ip = '127.0.0.1'
db_name = 'postgres'
#schema = 'db_projeto_cinema'
#print(f'{db_server}://{user}:{password}@{ip}/{db_name}')
engine = db.create_engine(f'{db_server}://{user}:{password}@{ip}/{db_name}')
conn = engine.connect()

NameError: name 'senha_pessoal' is not defined

In [794]:
df_valores_organizada.to_sql(name='TEMP_FILMES_DADOS', con=conn, if_exists='append', index=True, schema ='db_projeto_cinema' )

In [ ]:
# Fazer as consultas no API do IMDb
def filmes_imdb_dados(imdb_id,your_id):
    url = "https://movie-database-imdb-alternative.p.rapidapi.com/"

    querystring = {"r":"json","i": imdb_id }

    headers = {
        'x-rapidapi-host': "movie-database-imdb-alternative.p.rapidapi.com",
        'x-rapidapi-key': your_id
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    print(response.text)

your_id = 'b113a86e0emsha1d4031b4a27d5cp1ad48bjsn1bb87138d2ac'

filmes_imdb_dados('tt0499549',your_id)

Enviar os dados para PG

In [798]:

conexion1 = psycopg2.connect(database="bd1", user="postgres", password="heladera")
cursor1=conexion1.cursor()
sql="insert into articulos(descripcion, precio) values (%s,%s)"
datos=("naranjas", 23.50)
cursor1.execute(sql, datos)
datos=("peras", 34)
cursor1.execute(sql, datos)
datos=("bananas", 25)
cursor1.execute(sql, datos)
conexion1.commit()
conexion1.close()

def bulkInsert(records):
    
    try:
        connection = psycopg2.connect(user="postgres",
                                      password=senha_pessoal,
                                      host="127.0.0.1",
                                      port="5432",
                                      database="postgres",
                                      schema = "db_projeto_cinema")
        print(connection)
        cursor = connection.cursor()
        sql_insert_query = '''
                        INSERT INTO TB_DATA (data_aaaammdd, dia_semana, feriado, estacao, semana_do_mes) 
                           VALUES (%s,%s,%s,%s,%s)
                           '''

        # executemany() to insert multiple rows
        result = cursor.executemany(sql_insert_query, records)
        connection.commit()
        print(cursor.rowcount, "Record inserted successfully into mobile table")
        
    except (Exception, psycopg2.Error) as error:
        print(cursor)
        print("Failed inserting record into mobile table {}".format(error))

    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

OperationalError: connection to server on socket "/tmp/.s.PGSQL.5432" failed: FATAL:  database "bd1" does not exist


In [ ]:
#desempacotar o erro:
desempacotado_link = []
for i in lista_link_release_id_dia:
    print(i)
    print('\n')
    for j in i:
        print(j)
        desempacotado_link.append(j)

#df_link_por_dia['release_link'] = desempacotado_link

In [ ]:
url_do_filme = f'https://www.boxofficemojo.com/release/{link_id_filme_boxofficemojo}' # verificar como parametrizar esta parteresponse = requests.get(url_do_dia)
print(url_do_filme)
html=response.content
time.sleep(10)
soup = BeautifulSoup(html)
print(soup.prettify())

table_raw1 = soup.find('div', attrs = {'a-section mojo-body aok-relative'})
table_raw2 = table_raw1.find('div', attrs = {'a-section a-spacing-none mojo-gutter mojo-summary-table'})

print(table_raw1.prettify())


#a-section a-spacing-none mojo-performance-summary
#table_raw

#dados_domestico =  table_raw.find('span')

#print(dados_domestico.prettify())

'''

table_left = table_raw.find('div',attrs={'class': 'a-section mojo-body aok-relative'})
print(table_left.prettify())
                                          
table_raw = soup.find('div', attrs={'class': 'class="a-section a-spacing-none mojo-gutter mojo-summary-table"'})
#print(table_raw.prettify())
table_raw1 = table_raw.find('div', attrs={'class': 'a-section imdb-scroll-table-inner'})
#print(table_raw1.prettify())
table_rows = table_raw1.find_all('tr')
#print(table_rows[1].prettify())

for k in table_rows:
    link_desejado = extrair_link_top1(k)
    lista_link_release_id_dia.append(link_desejado)

print (table_row)
'''

'''
<div class="a-section a-spacing-none mojo-gutter mojo-summary-table"><div class="a-section a-spacing-none mojo-performance-summary">
'''